In [ ]:
pip install torch torchvision torchaudio --index-url https://download.pytorch.org/whl/cu118


In [ ]:
pip install tiktoken


In [ ]:
import whisper
import torch

# 디바이스 설정
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
# 모델 불러오기
model = whisper.load_model("turbo") 

# (1) 음성 파일 이름으로 바로 입력
result = model.transcribe("/home/ehost1/YS/Dataset/Old/노인남여_노인대화70_M_jang0972_60_수도권_실내_76454.wav")

# (2) 음성 파일을 불러들여서 텐서 형태로 입력
#audio = whisper.load_audio("/home/ehost1/YS/Dataset/Old/노인남여_노인대화70_M_jang0972_60_수도권_실내_76450.wav", sr=16000)
#result = model.transcribe(audio, fp16=False, language='ko')

# 결과 확인해보기
result['text']

In [ ]:
import whisper
import torch
import os
import time
import numpy as np

# 디바이스 설정 (GPU 사용 가능 여부에 따라)
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

# 사용할 모델 리스트
models = ['small', 'turbo', 'large']

# 테스트할 음성 파일들이 있는 폴더 경로 설정
audio_folder = '/home/ehost1/YS/Dataset/Sample/'
audio_files = [f for f in os.listdir(audio_folder) if f.endswith('.wav')]

# 결과를 저장할 디렉토리
output_dir = '/home/ehost1/YS/Whisper_results/'
os.makedirs(output_dir, exist_ok=True)

# 결과 저장용 딕셔너리
model_times = {model: [] for model in models}
model_texts = {model: [] for model in models}

# 모델마다 번갈아가며 테스트
for model_name in models:
    # 모델 불러오기
    model = whisper.load_model(model_name).to(device)
    print(f"Testing with model: {model_name}")
    
    # 각 음성 파일에 대해 처리
    for audio_file in audio_files:
        audio_path = os.path.join(audio_folder, audio_file)
        
        # 처리 시간 측정 시작
        start_time = time.time()
        
        # 음성 파일 처리 및 텍스트 변환
        result = model.transcribe(audio_path)
        
        # 처리 시간 기록
        processing_time = time.time() - start_time
        model_times[model_name].append(processing_time)
        
        # 결과 텍스트 저장
        result_text = result['text']
        output_file = os.path.join(output_dir, f"{model_name}_{audio_file}.txt")
        
        with open(output_file, 'w', encoding='utf-8') as f:
            f.write(result_text)
        
        model_texts[model_name].append(result_text)
        
    # 각 모델별 평균 처리 시간 출력
    avg_time = np.mean(model_times[model_name])
    print(f"Model: {model_name} | Average Time: {avg_time:.2f} seconds")

# 모델별 결과 비교 (평균 시간)
for model_name in models:
    avg_time = np.mean(model_times[model_name])
    print(f"Average time for model {model_name}: {avg_time:.2f} seconds")

# 각 모델의 텍스트 결과 저장을 위한 디렉토리
summary_dir = '/home/ehost1/YS/Whisper_summary/'
os.makedirs(summary_dir, exist_ok=True)

# 모델별 텍스트 결과를 summary 파일로 저장
for model_name in models:
    summary_file = os.path.join(summary_dir, f"{model_name}_summary.txt")
    with open(summary_file, 'w', encoding='utf-8') as f:
        for text in model_texts[model_name]:
            f.write(text + "\n")
    
    print(f"Text results for model {model_name} saved to {summary_file}")
